In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
import gdown
import os
from matplotlib.backends.backend_pdf import PdfPages
import shutil
from google.colab import drive

# Percorso alla cartella locale
drive_path = './'

# Scarico i file necessari da GitHub nel percorso locale
!wget -O "Lettura_Dati.py" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
!wget -O "background_singoloframe.py" https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/background_singoloframe.py

# controllo che i file siano stati scaricati correttamente
assert os.path.exists('Lettura_Dati.py'), "Lettura_Dati.py non trovato!"
assert os.path.exists('background_singoloframe.py'), "background_singoloframe.py non trovato!"

# importo i moduli dal percorso attuale
import sys
sys.path.insert(0, drive_path)
from background_singoloframe import Background_unFrame
from Lettura_Dati import leggi_file

# carico il background di un frame
bg = Background_unFrame()
background = np.array(bg)
print("Dimensione background:", background.size)

# Link con l'ID del file su Google Drive
file_url = "https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD"
# scarico il file da Google Drive
gdown.download(file_url, 'dati.txt', quiet=False)

# lettura file
file_path = "dati.txt"
frames = leggi_file(file_path)
frames_array = np.array(frames)

# calcolo lo spettrogramma
spectrogram = np.zeros((len(frames_array), 256 // 2), dtype=float)
for i, frame in enumerate(frames_array):
    yf_high = fft(frame[0:256], axis=0)
    spectrogram[i, :] = np.abs(yf_high[:256 // 2])

# parametri di tempo e frequenza
T_frame = 1
tempo_totale = len(frames_array) * T_frame
xf = fftfreq(256, T_frame / 256)[:256 // 2]
t = np.linspace(0, tempo_totale, len(frames))

# viene rimosso il background
spectrogram_without_baseline = spectrogram - background[:256 // 2]
spectrogram_without_baseline_real = np.real(spectrogram_without_baseline)

# seleziono le frequenze da 0 a 30 Hz
freq_min = 0
freq_max = 30
idx_freq_min = np.argmax(xf >= freq_min)
idx_freq_max = np.argmax(xf >= freq_max)
amplitudes_selected = spectrogram_without_baseline_real[:, idx_freq_min:idx_freq_max]
mean_ampl = np.mean(amplitudes_selected, axis=0)

# creo un PDF per salvare i grafici
pdf_filename = "grafici_outputM1.pdf"
with PdfPages(pdf_filename) as pdf:
    # Primo grafico: Media su N Dati con Frequenze (0-30 Hz)
    plt.figure(figsize=(10, 6))
    plt.plot(xf[idx_freq_min:idx_freq_max], mean_ampl)
    plt.xlabel('Frequenza [Hz]')
    plt.ylabel('Ampiezza')
    plt.title('Media su N Dati con Frequenze (0-30 Hz)')
    plt.grid(True)
    pdf.savefig()
    plt.close()

    # Secondo grafico: Spettrogramma in 2D
    log_spectrogram = np.log10(np.abs(spectrogram_without_baseline_real) + 1e-10)
    plt.figure(figsize=(12, 6))
    plt.imshow(log_spectrogram[:, idx_freq_min:idx_freq_max].T, aspect='auto',
               extent=[0, tempo_totale, freq_min, freq_max], origin='lower')
    plt.colorbar(label='Log(Ampiezza [au])')
    plt.xlabel('Tempo [s]')
    plt.ylabel('Frequenza [Hz]')
    plt.title('Spettrogramma (Frequenze da 0 a 30 Hz)')
    pdf.savefig()
    plt.close()

# carico il PDF su google drive
drive.mount('/content/drive', force_remount=True)
percorso_su_drive = '/content/drive/Shareddrives/Radar/Vital parameter monitoring/Sense2GoL_pulse_data/FileOutput/'
os.makedirs(percorso_su_drive, exist_ok=True)
shutil.move(pdf_filename, os.path.join(percorso_su_drive, pdf_filename))
print("File caricati con successo su Google Drive nel drive condiviso Radar!")


--2024-11-09 10:14:17--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/Lettura_Dati.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2371 (2.3K) [text/plain]
Saving to: ‘Lettura_Dati.py’

Lettura_Dati.py     100%[===================>]   2.32K  --.-KB/s    in 0s      

2024-11-09 10:14:17 (29.8 MB/s) - ‘Lettura_Dati.py’ saved [2371/2371]

--2024-11-09 10:14:17--  https://raw.githubusercontent.com/microlab-unibg/Sense2Gol-Pulse-vital-Parameters/Caricamento-file-e-collegamento-al-drive/MetodiDiRimozione/background_singoloframe.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to ra

Downloading...
From: https://drive.google.com/uc?id=1xIY_GuS68ld8mQzPV4w1YQaiYO4vqScq
To: /content/dati.txt
100%|██████████| 772k/772k [00:00<00:00, 24.0MB/s]


Dimensione background: 255


Downloading...
From: https://drive.google.com/uc?id=15LSYaKYgg9GrVJgwSE_dBTPDefqShAaD
To: /content/dati.txt
100%|██████████| 771k/771k [00:00<00:00, 79.3MB/s]


Mounted at /content/drive
File caricati con successo su Google Drive nel drive condiviso Radar!
